# Practical Aspects of Scientific Computing

> Your time is more valuable than computer time -- Everyone, ever

We discuss several aspects of computing that are things you do but don't often think about.  They can potentially save a large amount of time for you (not your computer).  In particular, we are going to look at

* **Versioning** -- maintaining a consistent set of code with history
* **Style** -- how you write your code influences its legibility
* **Debugging** -- finding and fixing problems in your code
* **Structure** -- the structure of your code
* **Tools** -- choosing the right set of tools for the job

# Versioning

Hard drive space is cheap, your time is not.  As you work, keep track of changes and help others work with you on your project.  One of the easy ways to do this is by using a Version Control System (VCS).  Popular ones include:

* **git** -- made by Linus T., distributed, light-weight branching, GitHub
* **hg** -- mercurial, similar to subversion also popular
* **CVS** -- older, branching is a little harder

Let's look a bit at *git*.  Git is a distributed version control system where each member has a full code history.  Each time a change is made, it is inserted into the history as a *commit*.  These commits can be grouped into larger structures known as *branches* which are off-shoot from a particular point in time.  They often contain a particular feature implementation or a bug fix. The simplest workflow for a single branch, single developer is this:

    git clone <resource-url>
    < edit some files >
    git add filename.ext
    git commit -m "an informative message about what you did"
    git push origin master
    
I am personally a big fan of [this workflow](http://nvie.com/posts/a-successful-git-branching-model/) which maintains a release oriented `master` branch, a `develop` branch which has active development, and `feature` branches for larger changes.  Additionally, this workflow includes `--no-ff` which keeps full branch history around after merges happen. That is, when done with a feature branch:

    git checkout -b feature
    < make some changes >
    git add <files>
    git commit -m "good, descriptive message"

    git checkout develop
    git merge --no-ff feature
    git branch -d feature
    git push origin develop
    
When working along a single branch I recommend the rebase workflow to allow for easier bisecting to find bugs.  In particular, when working with more than one developer, you may find that they have incorporated changes in between your syncs (via `fetch` or `pull`).  To remedy this, I personally do:

    git push origin master
         ![rejected] (someone else has pushed and you are not synced)
         
    git fetch origin
    git rebase origin/master
    git push origin master

This keeps the history mostly linear and makes it easier to track changes.

![](git-branches.png)

# Style

Being able to sit down and understand a code base quickly is important.  To make sure you play well with others, style is important.  Different companies, languages, and individuals have preferred styles.  Python is nice in that there is a well-defined style guide that everyone can use: [PEP8](https://www.python.org/dev/peps/pep-0008/). Some of the suggestions are a little out-dated so take with a grain of salt, but here are a few key points about formatting -

* Spaces are preferred (4 to a tab)
* Shorter lines are easier to read (when completed correctly)
* Avoid extra whitespace
* No bare `except`s


Naming conventions 
* **`CamelCase`** for classes
* **`underscore_lowercase`** for functions and variables (`mixedCase` okay if prevailing style)
* **`oneword`** for modules
* **`CAPITAL_WORDS`** for constants

Comments
* `"""` blockquotes for docstrings
* `#` for multiline and (occasional) inline comments

Also, it is important to stay 'Pythonic'. Prefer list / dictionary comprehensions for shorter statements, keep ideas separated using whitespace.  Comment in blocks unless necessary. Use parallel structure in code when it is necessary. For example, consider the following three code blocks (NOTE, I did not check to make sure these work):

In [7]:
def lowpass_filter_3d(field, freq_cutoff):
    """
    Performs a low-pass filter on 3D `ndarray`, removing frequencies
    higher than `freq_cutoff` where a frequency is defined by `np.fft.fftfreq`
    
    Parameters:
    -----------
        field : 3D ndarray [M,N,P]
            The field which we wish to perform the filter on, has 3 dimensions
            
        freq_cutoff : float
            Hard frequency cutoff for the filter, frequencies are defined by
            the FFT standard implemented in `np.fft.fftfreq`
    
    Returns:
    --------
        filtered_field : 3d ndarray [M,N,P]
            The filtered, real-space field after applying the low-pass filter
    """
    # take the FFT of the incoming data
    qfield = np.fft.fftn(field)

    # build an array of frequencies for each axis dimension,
    # broadcast them using a meshgrid, then taking the norm along the last axis
    kval = [np.fft.fftfreq(i) for i in field.shape]
    kvec = np.meshgrid(*kval, indexing='ij')
    klen = np.sqrt((kvec**2).sum(axis=-1))

    # zero the frequencies beyond our cutoff, inverse FFT and real part
    qfield[klen > freq_cutoff] = 0
    return np.real(np.fft.ifftn(qfield))

def LowpassFilter3d(i, freq):
    """ Perform low pass filter """
    a = np.fft.fftn(i) # take the FFT of our data
    kx, ky, kz = np.fft.fftfreq(a.shape[0]), np.fft.fftfreq(a.shape[1]), np.fft.fftfreq(a.shape[2]) # create the kvectors for this particular array shape
    k = np.sqrt(np.sum(kx[:,None,None]**2 + ky[None,:,None]**2 + kz[None,None,:]**2, axis=-1))
    qout = np.where(k > freq, [np.zeros_like(k), qfield])
    rout = np.real(np.fft.ifftn(qout))
    return rout

def lowpass_filter_3d(field, freq_cutoff):
    """ low pass filter
    """
    # take the fftn of the field
    qfield = np.fft.fftn(field)

    # create the kx array
    kx = np.fft.fftfreq(a.shape[0])
    
    # create the ky array
    ky = np.fft.fftfreq(a.shape[1])
    
    # create the kz array
    kz = np.fft.fftfreq(a.shape[2])
    
    # create the length of the frequency vector
    klen = np.sqrt(np.sum(kx[:,None,None]**2 + ky[None,:,None]**2 + 
        kz[None,None,:]**2, axis=-1))

    # zero the frequencies beyond our cutoff, inverse FFT and real part
    qfield[klen > freq_cutoff] = 0
    out = np.real(np.fft.ifftn(qfield))
    # return the result
    return out

# Debugging

One of the most common things you do when creating a software package or solving a science problem is to debug your code.  I'm taking 'debug' in the sense of removing problems from your code.  So the natural place to begin is: how do you find bugs?

## Finding bugs / Tests

While the most natural way to find problems is to allow them to surface, this is not always the best practice.  Some issues are more subtle than you'd expect / appreciate at first glance.  That is why some people advocate for *test-driven* development. In this paradigm, you must write a battery of tests that cover the entire functionality of the code base.  

In [1]:
import matplotlib.pylab as pl
import numpy as np
import scipy as sp
%matplotlib

Using matplotlib backend: Agg


#### Example: chaotic scattering

Chaos is very generic property of dynamical systems usually characterized by sensitivity to initial conditions, topological mixing, and many dense periodic orbits.  For example if you consider the path of light through a set of reflective balls, the trajectory of the light is very complex due to the non-analytic edges of the objects.  The image from such a setup creates a real-space fractal as seen below.  In fact, each color in the picture touches every other color even at their boundary (?!).

![](wada2.jpg)

Christmas ball fractal (src: http://www.evilmadscientist.com/2006/christmas-chaos/)

To study this system, we essentially need to propogate light by finding the intersection of lines and circles (for th 2D case).  Such a function may look like:

In [2]:
def sgn(a):
    """
    Return the sign of the argument (in contrast to np.sign)
        -1 if a < 0
        +1 if a >= 0
        
    Parameters:
    -----------
        a : int, float
            the argument to check
    
    Examples:
    
    >>> sgn(-1)
    -1.0
    
    >>> sgn(0)
    1.0

    >>> sgn(1e9)
    1.0
    """
    return -1.0*(a < 0) + 1.0*(a >= 0)

def intersection_line_circle(pt0, pt1, radius):
    """
    Find the intersection between line defined by points pt0 and pt1
    and a circle of size radius at the origin (0,0) given by quadratic
    solution (see http://mathworld.wolfram.com/Circle-LineIntersection.html).
    
    Parameters:
    -----------
        pt0 : array_like -> [2]
            Beginning point for line of the form [x,y]

        pt1 : array_like -> [2]
            End point for line of the form [x,y]

        radius : float
            The radius of the circle at the origin (0,0) to intersect

    Returns:
    --------
        intersection : array_like
            A list of the intersections for the line with the circle,
            Can be as large as [2,2] and small as empty.  If no intersections,
            will return empty list.  Two intersections will be returned
            in the form [[x1, y1], [x2, y2]]

    Note: since this is example code, I will not be respecting floating-point
          issues introduced by loss of significance by quadratic formula
          (see https://en.wikipedia.org/wiki/Loss_of_significance)

    Examples:

    >>> intersection_line_circle([1.0, 0.0], [1.0, 1.0], 0.5)
    array([], dtype=float64)

    >>> intersection_line_circle([1.0, 0.0], [1.0, 1.0], 1-1e-14)
    array([], dtype=float64)

    >>> intersection_line_circle([0.5, 0.5], [0.0, 0.5], 0.5)
    array([[ 0. ,  0.5]])

    >>> intersection_line_circle([0.0, 1.0], [1.0, 0.0], np.sqrt(2)/2)
    array([[ 0.5,  0.5]])

    >>> intersection_line_circle([0.1, 0.0], [1.0, 0.0], 1)
    array([[ 1.,  0.],
           [-1., -0.]])

    >>> intersection_line_circle([0.0, 0.0], [0.0, 1.0], 1)
    array([[ 0.,  1.],
           [ 0., -1.]])

    >>> intersection_line_circle([0.0, 0.0], [1.0, 1.0], 1)
    array([[ 0.70710678,  0.70710678],
           [-0.70710678, -0.70710678]])
    """

    # convert to ndarray if not already
    if not isinstance(pt0, np.ndarray):
        pt0 = np.array(pt0).astype('float')
    if not isinstance(pt1, np.ndarray):
        pt1 = np.array(pt1).astype('float')

    # get the differences between positions
    dx,dy = pt1 - pt0

    # calculate parts of the formula given by the reference
    r2 = radius*radius
    dr2 = dx*dx + dy*dy
    D = np.cross(pt0, pt1)

    # calculate the descriminant telling how many intersections
    dsc = r2*dr2 - D*D

    # if the descriminant is less than zero, return appropriate nothings
    if dsc < 0:
        return np.array([])

    elif np.allclose(dsc, np.finfo(float).eps):
        x0 = D*dy/dr2
        y0 = -D*dx/dr2
        return np.array([[x0, y0]])

    else:
        x0 = (D*dy + sgn(dy)*dx*np.sqrt(dsc)) / dr2
        y0 = (-D*dx + np.abs(dy)*np.sqrt(dsc))/dr2

        x1 = (D*dy - sgn(dy)*dx*np.sqrt(dsc)) / dr2
        y1 = (-D*dx - np.abs(dy)*np.sqrt(dsc))/dr2

        return np.array([[x0, y0], [x1, y1]])

### Test framework 1: unittest

Create a series of tests to cover every possible return case and extreme values to make sure that everything is working well.  

In [3]:
import sys
import unittest

class TestSignFunction(unittest.TestCase):
    def test_neg0(self):
        self.assertEqual(sgn(-1.0), -1)

    def test_neg1(self):
        self.assertEqual(sgn(-1e9), -1)

    def test_neg2(self):
        self.assertEqual(sgn(-1e-14), -1)

    def test_zero(self):
        self.assertEqual(sgn(0), 1)

    def test_pos0(self):
        self.assertEqual(sgn(1e9), 1)
        
    def test_pos1(self):
        self.assertEqual(sgn(1), 1)
                
class TestLineCircleIntersection(unittest.TestCase):
    def check(self, pt0, pt1, rad, length, value=None):
        out = intersection_line_circle(pt0, pt1, rad)
        self.assertEqual(len(out), length)
        if len(out) > 0:
            assert np.allclose(out, value)

    def test_no_intersect_far(self):
        self.check([1.0, 0.0], [1.0, 1.0], 0.5, 0)
        self.check([1.0, 0.0], [1.0, 1.0], 0.9, 0)

    def test_no_intersect_near(self):
        self.check([0.95, 0.2], [1.0, 1.0], 0.9, 0)

    def test_no_intersect_infinitessimal(self):
        self.check([1.0, 0.0], [1.0,1.0], 1-1e-14, 0)

    def test_one_intersect0(self):
        self.check([0.5, 0.5], [0.0, 0.5], 0.5, 1, np.array([[0.0, 0.5]]))

    def test_one_intersect1(self):
        self.check([0.0, 1.0], [1.0, 0.0], np.sqrt(2)/2, 1, np.array([[0.5,0.5]]))

    def test_two_intersect0(self):
        self.check([0.1, 0.0], [1.0, 0.0], 1, 2, np.array([[1.0,0.0],[-1.0,0.0]]))

    def test_two_intersect1(self):
        self.check([0.0, 0.0], [0.0, 1.0], 1, 2, np.array([[0.0,1.0],[0.0,-1.0]]))

    def test_two_intersect2(self):
        rt2 = np.sqrt(2.0)/2.0
        self.check([0.0, 0.0], [1.0, 1.0], 1, 2, np.array([[rt2, rt2], [-rt2, -rt2]]))

suite0 = unittest.TestLoader().loadTestsFromTestCase( TestSignFunction )
suite1 = unittest.TestLoader().loadTestsFromTestCase( TestLineCircleIntersection )
unittest.TextTestRunner(verbosity=1,stream=sys.stderr).run( suite0 )
unittest.TextTestRunner(verbosity=1,stream=sys.stderr).run( suite1 )

......
----------------------------------------------------------------------
Ran 6 tests in 0.002s

OK
........
----------------------------------------------------------------------
Ran 8 tests in 0.015s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

### Test framework 2: doctest

Keep your documentation in your code directly.  In your docstring you can include lines that look like:

    >>> sgn(-1.0)
    1.0
    
Here, the `>>>` means to run this line and the next line is the expected output (as a string).  You can then run all documented tests with 

    import doctest
    doctest.testmod()
    
### Test framework N

There are many other frameworks each with their pros and cons.  I've heard good things about nose, and py.test (very mixed here) but there is a very large number to [choose from](https://wiki.python.org/moin/PythonTestingToolsTaxonomy)

## Resolving Bugs

After you have your tests set up, you will quickly find what is working and what isn't (aside from those side cases that you hadn't found before).  Now that you have a problem, how do you fix it?

First, you should 

Tests
Simplest case
Bisect print
ipdb, %debug, Tracer()
break points, old school